### WorkFlow (Spiceworks -> OsTicket)

Author:   Sierra Bonilla
<br></br>
Date:     04-08-2023
<br></br>


      FOR SW_TICKET_ID IN ALL_SW_TICKET_IDS:

      --------------------- SPICEWORKS EXPORT ---------------------

         1. TD = GET_TICKET(SW_TICKET_ID)
         
         2. UD = GET_USER(TD['ASSIGNED_TO'])
         
         3. CD = GET_COMMENT(SW_TICKET_ID)

      ----------------------- OSTICKET IMPORT -----------------------

         4. OS_USER_ID = CREATE_USER_IF_NOT_EXISTS(USER_DETAILS)

         5. BUILD TICKET_DETAILS DICTIONARY 

         6. OS_TICKET_ID = CREATE_TICKET(TICKET_DETAILS)

         7. BUILD COMMENT_DETAILS DICTIONARY 

         8. OS_THREAD_ID = CREATE_COMMENT(COMMENT_DETAILS)

##### Import Functions/Libraries

In [1]:
from get_ticket import get_ticket
from get_user import get_user
from get_comments import get_comments
from create_ticket_helper_fns import check_key_in_json_file
from create_user import create_user_if_not_exists
from create_ticket import create_ticket
from create_comment import create_comment

##### Define Global Variables & File Paths

In [2]:
ticket_id = 30
hostname = 'spiceworks'
database = 'osticket-clone'

staff_id_file = r'dictionaries\staff_email_to_id.txt'
staff_username_file = r'dictionaries\staff_email_to_un.txt'
spe_file = r'dictionaries\sw_spe_to_os_spe.txt'
spe_id_file = r'dictionaries\sw_spe_to_os_spe_id.txt'
dep_file = r'dictionaries\sw_dep_to_os_dep.txt'
dep_id_file = r'dictionaries\sw_dep_to_os_dep_id.txt'

import_bot_id = 11
import_bot_username = "irobot"
import_bot_name = "Import Bot"

##### **1. SPICEWORKS EXPORT:** Get Ticket Details

In [3]:
sw_ticket_details = get_ticket(ticket_id)
sw_ticket_details

{'id': 30,
 'summary': 'Access to the cancer share drive',
 'description': 'Hello\n\nI’m not sure this is the correct group to contact but I need access to the Cancer share drive. Is this something you could help with?\n\nKind regards\n\nSteph Karis – Senior Radiotherapy Dosimetrist\n\nRadiotherapy Physics Department\n\nDirect line telephone:020 3447 7732\n\nWe are committed to delivering top-quality patient care, excellent education and world class research\n\nsafety kindness teamwork improving\n\nUCLH website | Media | Become a member | Support our charities | Contact UCLH\n\nFind us on Facebook | Follow us on Twitt',
 'created_at': '2020-04-30 13:25:27',
 'closed_at': '2020-05-01 16:12:01',
 'created_by': 's.karis@nhs.net',
 'assigned_to': 'david.taylor4@nhs.net',
 'spe': 'SHARE DRIVE',
 'department': 'RADIOTHERAPY'}

##### **2. SPICEWORKS EXPORT:** Get User Details

In [4]:
sw_user_details = get_user(sw_ticket_details['created_by'])
sw_user_details

{'id': 19, 'email': 's.karis@nhs.net', 'name': None, 'phone': None}

##### **3. SPICEWORKS EXPORT:** Get Comment Details

In [5]:
sw_comment_details = get_comments(ticket_id)
sw_comment_details

{'id': 30,
 'body': '<h2>From Spiceworks</h2><h3><code>david.taylor4@nhs.net</code> on 2020-05-01 16:12:01:</h3><p>Ticket closed: sent email to user, advised would need to request Cancer Share access through Atos/ServiceNow as MedPhys cannot provision</p><hr><h3><code>david.taylor4@nhs.net</code> on 2020-04-30 17:35:59:</h3><p>Assigned to David Taylor.</p><hr><h3><code>s.karis@nhs.net</code> on 2020-04-30 13:25:27:</h3><p>Attachment:</p><p><em>image001.png</em></p><hr>Attachments folder: <code>\\\\spiceworks\\Ticket\\30</code>'}

##### **4. OSTICKET IMPORT:** Create User 

In [6]:
os_user_result = create_user_if_not_exists(sw_user_details, hostname=hostname, database=database)
if os_user_result[0]:
    print('User {} was created with id {}'.format(sw_user_details["email"], os_user_result[1]))
else:
    print('User {} already exists with id {}'.format(sw_user_details["email"], os_user_result[1]))

User s.karis@nhs.net already exists with id 165


##### **5. OSTICKET IMPORT:** Build Ticket Details 

##### Ticket Detail Conversion Table

|sw_ticket_details   |os_ticket_details      |
|--------------------|-----------------------|
| 'id'               | 'id' (spiceworks id num)|
| 'summary'          | 'summary'             |
| 'description'      | 'description'         |
| 'created_at'       | 'created_at'          |
| 'closed_at'        | 'closed_at'           |
| 'created_by' (email)| 'created_by' (osTicket user id)|
|                    | 'created_by_email'    |
| 'assigned_to'  (email)| 'assigned_to' (osTicket staff id)|
|                    | 'assigned_to_username'|
| 'spe' (spiceworks spe)| 'spe' (osTicket spe)|
|                    | 'spe_id' (osTicket spe ID)|
| 'department' (spiceworks dept.)| 'department' (osTicket dept.)|
|                    | 'department_id' (osTicket dept. id)

In [7]:
os_ticket_details = {}
os_ticket_details["id"] = sw_ticket_details["id"]
os_ticket_details["summary"] = sw_ticket_details["summary"]
os_ticket_details["description"] = sw_ticket_details["description"]
os_ticket_details["created_at"] = sw_ticket_details["created_at"]
os_ticket_details["closed_at"] = sw_ticket_details["closed_at"]
os_ticket_details["created_by"] = os_user_result[1]
os_ticket_details["created_by_email"] = sw_user_details["email"]

`assigned_to`

In [8]:
assigned_to_results = check_key_in_json_file(sw_ticket_details["assigned_to"], staff_id_file)
if assigned_to_results[0]:
    os_ticket_details["assigned_to"] = assigned_to_results[1]
    os_ticket_details["assigned_to_username"] = check_key_in_json_file(sw_ticket_details["assigned_to"], staff_username_file)[1]
else:
    os_ticket_details["assigned_to"] = import_bot_id
    os_ticket_details["assigned_to_username"] = import_bot_username 

`spe`

In [9]:
spe_results = check_key_in_json_file(sw_ticket_details["spe"], spe_file)
if spe_results[0]:
    os_ticket_details["spe"] = spe_results[1]
    os_ticket_details["spe_id"] = check_key_in_json_file(sw_ticket_details["spe"], spe_id_file)[1]
else:
    os_ticket_details["spe"] = "IMPORTED"
    os_ticket_details["spe_id"] = 46

`dept`

In [10]:
dept_results = check_key_in_json_file(sw_ticket_details["department"], dep_file)
if dept_results[0]:
    os_ticket_details["department"] = dept_results[1]
    os_ticket_details["department_id"] = check_key_in_json_file(sw_ticket_details["department"], dep_id_file)[1]
else:
    os_ticket_details["department"] = "IMPORTED"
    os_ticket_details["department_id"] = 34

In [11]:
os_ticket_details

{'id': 30,
 'summary': 'Access to the cancer share drive',
 'description': 'Hello\n\nI’m not sure this is the correct group to contact but I need access to the Cancer share drive. Is this something you could help with?\n\nKind regards\n\nSteph Karis – Senior Radiotherapy Dosimetrist\n\nRadiotherapy Physics Department\n\nDirect line telephone:020 3447 7732\n\nWe are committed to delivering top-quality patient care, excellent education and world class research\n\nsafety kindness teamwork improving\n\nUCLH website | Media | Become a member | Support our charities | Contact UCLH\n\nFind us on Facebook | Follow us on Twitt',
 'created_at': '2020-04-30 13:25:27',
 'closed_at': '2020-05-01 16:12:01',
 'created_by': 165,
 'created_by_email': 's.karis@nhs.net',
 'assigned_to': 5,
 'assigned_to_username': 'dtaylor',
 'spe': 'IMPORTED',
 'spe_id': 46,
 'department': 'RADIOTHERAPY',
 'department_id': 40}

##### **6. OSTICKET IMPORT:** Create Ticket

In [14]:
os_ticket_result = create_ticket(os_ticket_details, hostname=hostname, database_name=database)
if os_ticket_result[0]:
    print('\nTicket was created & closed!')
    print('\U0001f984 Ticket ID: {}\n \U0001f984 Ticket Number: {}\n'.format(os_ticket_result[1], os_ticket_details["id"]))
else:
    print('\nTicket already exists! \n\U0001f984 Ticket Number: {}'.format(os_ticket_details["id"]))
    print('\U0001f984 Ticket ID: {}\n'.format(os_ticket_result[1]))


Ticket already exists! 
🦄 Ticket Number: 30
🦄 Ticket ID: 143



In [17]:
os_ticket_id = os_ticket_result[1]

##### **7. OSTICKET IMPORT:** Build Comments Dictionary



##### Comment Detail Conversion Table

|sw_comment_details|os_comment_details|
|------------------|------------------|
| 'id'             | 'ticket_id' (ost_ticket id)    |
| 'body'           | 'body_html'      |
|                  | 'staff_name'     |
|                  | 'title'          |
|                  | 'staff_id'       |

In [18]:
os_comment_details = {}
os_comment_details["ticket_id"] = os_ticket_id
os_comment_details["body_html"] = sw_comment_details["body"]
os_comment_details["staff_name"] =  import_bot_name
os_comment_details["title"] = "All Replies/Comments"
os_comment_details["staff_id"] = 11

In [19]:
os_comment_details

{'ticket_id': 143,
 'body_html': '<h2>From Spiceworks</h2><h3><code>david.taylor4@nhs.net</code> on 2020-05-01 16:12:01:</h3><p>Ticket closed: sent email to user, advised would need to request Cancer Share access through Atos/ServiceNow as MedPhys cannot provision</p><hr><h3><code>david.taylor4@nhs.net</code> on 2020-04-30 17:35:59:</h3><p>Assigned to David Taylor.</p><hr><h3><code>s.karis@nhs.net</code> on 2020-04-30 13:25:27:</h3><p>Attachment:</p><p><em>image001.png</em></p><hr>Attachments folder: <code>\\\\spiceworks\\Ticket\\30</code>',
 'staff_name': 'Import Bot',
 'title': 'All Replies/Comments',
 'staff_id': 11}

##### **8. OSTICKET IMPORT:** Create Comment

In [20]:
os_comment_result = create_comment(os_comment_details, hostname=hostname, database_name=database)
if os_comment_result[0]:
    print('\nComment was created!')
    print('\U0001f984 Ticket ID: {}\n\U0001f984 Thread ID: {}\n'.format(os_comment_details["ticket_id"], os_comment_result[1]))
else:
    print('\nUnable to make comment! \n:( Ticket Number: {}\n'.format(os_comment_details["ticket_id"]))


Comment was created!
🦄 Ticket ID: 143
 🦄 Thread ID: 347

